In [1]:
import os, re
import pickle
opj = os.path.join
import numpy as np
import nibabel as nib 
from nilearn.plotting import view_img
import toolz

import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import HTML, Image, display
from matplotlib import pyplot as plt
%matplotlib inline

C:\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
C:\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.datasets.base module is  deprecated in version 0.22 and will be removed in version 0

In [2]:
LESION_MAPS_FILE = r'C:\Lab\Lesion Project\Maps\Spatial\FinalMaps.pic'
LESION_DATA = pickle.load(open(LESION_MAPS_FILE, 'rb'))
VOL_SIZE = LESION_DATA['volSize']
GM_INDICES = LESION_DATA['gmIndices']
SUBJECT_MAPS = {key:LESION_DATA[key] for key in ['Mean', 'Max']}

SOME_NMI_FILE = r'C:\Lab\Atlases\Brainnetome\BN_Atlas_246_3mm.nii.gz'
obj = nib.load(SOME_NMI_FILE)
NII_HEADER = obj.header
NII_HEADER.set_data_dtype(float)
NII_AFFINE = obj.affine

LESION_MASKS = pickle.load(open(r'C:\Lab\Lesion Project\Maps\Spatial\LesionMasks.pic', 'rb'))
SUBJECT_KEYS = [f'{i+1:02}' for i in range(68)]

In [3]:
def prepareNiiObject(sbjKey, aggKey, weighingKey):
    iSbj = int(sbjKey)-1
    sbjVxlValues = SUBJECT_MAPS[aggKey][weighingKey][:,iSbj]
    imgdata = np.zeros(np.prod(VOL_SIZE))
    imgdata[GM_INDICES] = sbjVxlValues
    imgdata = imgdata.reshape(VOL_SIZE)
    niiMapObj = nib.Nifti1Image(imgdata, NII_AFFINE, NII_HEADER)
    
    sbjMaskInds = LESION_MASKS[iSbj]
    imgdata = np.zeros(np.prod(VOL_SIZE))
    imgdata[sbjMaskInds] = 1
    imgdata = imgdata.reshape(VOL_SIZE)
    niiMaskObj = nib.Nifti1Image(imgdata, NII_AFFINE, NII_HEADER)
    
    return niiMapObj, niiMaskObj

In [4]:
htmlObjs = [0, 0]

@interact(sbjKey=SUBJECT_KEYS, aggKey=['Mean', 'Max'], threshold=(0,1,0.05))
def view(sbjKey, aggKey, threshold=0.2):
    niiMapObj, niiMaskObj = prepareNiiObject(sbjKey, aggKey, 'None')
    htmlObjs[0] = view_img(niiMapObj, threshold=threshold, vmax=1)
    htmlObjs[0].width = 900
    htmlObjs[0].height = 270*1.5
    htmlObjs[1] = view_img(niiMaskObj, threshold=0.9, vmax=1, resampling_interpolation='nearest')
    htmlObjs[0].width = 900
    htmlObjs[0].height = 270*1.5
    display(htmlObjs[0])
    display(htmlObjs[1])

interactive(children=(Dropdown(description='sbjKey', options=('01', '02', '03', '04', '05', '06', '07', '08', …